# Prerrequisitos 

## Dependencias 



In [ ]:
import jax.numpy as np 
from jax import random, jit, vmap 
from inrmri.data_harvard import load_data_without_bart
from inrmri.data_splitter import SimpleDataLoader
from inrmri.fourier_features import FF_fraction_static_mixed_net
from inrmri.radon_training import spacelim, spoke_loss_fourierspace_phase

from inrmri.basic_nn import simple_train 
import optax 

import matplotlib.pyplot as plt 

# semillas para reproducir el entrenamiento 

random_seed  = 0

key_split, key_B, key_init, key_train = random.split(random.PRNGKey(random_seed), 4)


## BART Toolbox 

Correr el ejemplo requiere tener instalado el [BART Toolbox](https://mrirecon.github.io/bart/). Ver las instrucciones el el [`README-es`](../README-es.md)

# Obtener datos y generar el conjunto de entrenamiento 

Los datos se obtendrán de [Replication Data for: Multi-Domain Convolutional Neural Network (MD-CNN) For Radial Reconstruction of Dynamic Cardiac MRI](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi%3A10.7910%2FDVN%2FCI3WB6&version=2.0&q=&fileAccess=&fileTag=&fileSortField=&fileSortOrder=). 

## Parámetros para el procesamiento de datos 

- `chosen_patient`:`str` en `"P01"`, `"P02"`, ..., `"P15"`. El código funciona con cualquiera que esté en el diccionario `HARVARD_DB_IDs` de `inrmri.data_harvard`. Es posible añadir pacientes extras añadiendo nuevas keys al diccionario. El _id_ pedido es único a cada paciente y es parte del URL de descarga como un código de 6 valores (letras o números) que está después del _id_ del repositorio (`CI3WB6`). Para [el paciente `P100`](https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/CI3WB6/HR66QD&version=2.0) por ejemplo, su _id_ es `HR66QD`.

- `sub_spokes_per_frame`:`int`. Relacionado con el factor de aceleración que se desea. Los datos disponible son (casi) totalmente muestreados (usualmente 196 _spokes_ disponibles para reconstruir una imagen de 208x208, lo que da un factor de aceleración muy bajo de $208/196 \approx 1.06$). Pero nos interesa hacer reconstrucciones submuestreadas, es decir, con muchas menos _spokes_ por frame. Usar 16 spokes por frame (`sub_spokes_per_frame=16`) es una cantidad razonable. Usar más debería dar mejores reconstrucciones (pero considerar que el tamaño del _training set_ es directamente proporcional a la cantidad de _spokes_ por frame, por lo que habría que aumentar el número de iteraciones para mantener el mismo número de _epochs_). Yo suelo usar 8, y también es posible usar menos, pero en general deja de tener sentido físico usar factores de aceleración tan grandes (usando menos de 8 spokes por frame, el total de spokes que se adquirirían prospectivamente no alcanza para un ciclo cardíaco completo).

- `CSMAP_FOLDER`: Directorio donde están almacenados los mapas de sensibilidad, con nombres de la forma `csmap-P01.npy` (`csmap-` + `'chosen_patient'` + `.npy`)

In [ ]:
chosen_patient = "P07"
sub_spokes_per_frame = 12
CSMAP_FOLDER = '/home/tabita/ACIP-MRI/ACIP-MRI/data_coils/'

## Carga de datos 

A partir del diccionario de configuración, se utiliza la función `load_data` para cargar todo lo necesario. La primera vez que se usa con un paciente descarga el dataset y lo guarda en una carpeta `harvardDB/` que debe haber sido creada previamente. También debe existir `data_bart/`, donde se guardarán las archivos generados por BART: 

- `X_full, Y_full`: conjunto de entrenamiento para la red. Al cargar los datos con `hermitic_fill=True` y `relaxed_pad_removal=False`, la dimensión de _read out_ (el largo de las _spokes_) es de 414. Cada spoke se trata como 1 dato (es decir `batch_size=2` corresponde a 2 spokes). El total de datos entonces es `sub_spokes_per_frame * total_frames`, donde `total_frames` es la cantidad total de frames en que están agrupados los datos. `X_full:array[float]` tiene la información de todas las _spokes_ resultantes luego del submuestreo (la ubicación de las frecuencias adquiridas en el k-space específicamente), además del _frame_ al que están asociadas. `Y_full:array[complex]` tiene las mediciones de cada _spoke_ para cada una de las bobinas.
  - `X_full.shape = (sub_spokes_per_frame * total_frames, 414 + 1, 2)`. El 1 del `414 + 1` es una forma fea de añadir el tiempo de la adquisición a los datos, y el 2 es porque las adquisiciones son 2D en el k-space ($k_x, k_y$).
  - `Y_full.shape = (sub_spokes_per_frame * total_frames, n_coils, 414, 1)`. `n_coils` es el número de bobinas con que se adquirieron los datos. El 1 es porque facilita ajustar los datos con la dimensión de salida de la red.
- `csmap:array[complex]`: mapas de sensibilidad estimados con BART a partir de los datos totalmente muestreados. `csmap.shape = (n_coils, 414, 414)` (el largo de la _spoke_ determina el tamaño de la imagen reconstruida).
- `im:array[complex]`: una reconstrucción de referencia (un vídeo) obtenida a partir de la adquisición totalmente muestreada y GRASP con parámetros $\lambda = 1\times10^{-3}, \mathcal{L} = 5\times 10^{-4}$ y 100 iteraciones. `im.shape = (414,414,total_frames)`. 
- `hollow_mask:array[float]`: usualmente los mapas de sensibilidad tienen zonas donde todas las bobinas tienen sensibilidad 0. Esto impide que red sea capaz de estimar una reconstrucción en esas zonas, porque no tienen ninguna importancia en la función de pérdida. Esto en general no importa, ya que esas zonas suelen estar fuera del cuerpo, así que es razonable que la estimación allí sea 0. En la práctica, se permite que la red prediga cualquier cosa en esas zonas y luego se usa la máscara para cubrirlas con 0s. `hollow_mask.shape = (414,414)` y *marca las zonas en las que no se pueden reconstruir*, es decir, vale 1 donde no se puede optimizar la red (la zona que se va a enmascarar posteriormente) y 0 donde sí se puede optimizar.


La función `get_splitted_dataset` me permite dejar algunas _spokes_ como conjunto de validación.

In [ ]:
X_full, Y_full, csmap, im, hollow_mask = load_data_without_bart(chosen_patient, sub_spokes_per_frame, CSMAP_FOLDER)

loader = SimpleDataLoader(X_full, Y_full)

training_fraction = 0.7 

X_train, Y_train, X_val, Y_val = loader.get_splitted_dataset(key_split, training_fraction)

trainset = (X_train, Y_train)
valset = (X_val, Y_val)

In [ ]:
# Visualizar la reconstrucción de referencia, los mapas de sensibilidad y la máscara

chosen_csmap = 12 
chosen_frame = 15 
fig = plt.figure(figsize=(15,5))

# TODO: hacer las recos 
# plt.subplot(131)
# plt.imshow(np.abs(im[...,chosen_frame]), cmap='bone')
# plt.title(f"Reconstrucción completamente, \nmuestreada en el {chosen_frame}-ésimo frame")

plt.subplot(132)
plt.imshow(np.abs(csmap[chosen_csmap]))
plt.title(f"{chosen_csmap}-ésimo mapa de sensibilidad")

plt.subplot(133)
plt.imshow(hollow_mask, cmap='Reds')
plt.title(f"Zonas donde la reconstrucción \n de la red está indeterminada")


## Cargar la red 

Por simplicidad voy a elegir la red tipo `FF_fraction_static_mixed_net`, que es la que ha dado mejores resultados. Esta red usa un vector de Fourier Features de tipo STiFF, descrito [en este paper](https://arxiv.org/pdf/2307.14363.pdf), que está dado por

$$
\gamma(\mathbf{x}, t; \beta) =
\begin{bmatrix}
\cos(2\pi B_{\text{s}}\mathbf{x}) \\
\sin(2\pi B_{\text{s}}\mathbf{x}) \\
\cos(2\pi B_{\text{d}}\mathbf{x})\cos(2 \pi t) \\
\cos(2\pi B_{\text{d}}\mathbf{x})\sin(2 \pi t) \\ 
\sin(2\pi B_{\text{d}}\mathbf{x})\cos(2 \pi t) \\
\sin(2\pi B_{\text{d}}\mathbf{x})\sin(2 \pi t)
\end{bmatrix} \in \mathbb{R}^{2M_{\text{s}} + 4M_{\text{d}}}
$$

Cada componente de $B_s \in \mathbb{R}^{M_s \times 2}$ y $B_d \in  \mathbb{R}^{M_d \times 2}$ viene de una normal $\mathcal{N}(0,\sigma)$. El largo del vector STiFF es $L = 2M_s + 4M_d$. Se define $p_s:= \frac{2M_s}{L}$ como la fracción de componentes del vector STiFF que no dependen del tiempo.


- `sigma:float`: parámetro $\sigma$, relacionado con las frecuencias que la red aprende más rápidamente. Para este dataset, usualmente valores de $\sigma \in [3, 12]$ dan buenos resultados.
- `desired_ffvector_len:int`: Aproximadamente $L$, el largo del vector de Fourier Features. En general, mientras más grande, más robusto se hacen los resultados al valor de `sigma`, pero también requiere más memoria.
- `static_fraction:float`: parámetro $p_s \in [0,1]$ asociado a la regularización temporal: mientras más cercano a 1, mayor regularización. Depende del submuestreo, mientras más submuestreo (menos datos, menor `sub_spokes_per_frame`) se necesita más regularización. Para `sub_spokes_per_frame=8`, `static_fraction` entre 0.65 y 0.8 funciona bien.

In [ ]:
FFnet = FF_fraction_static_mixed_net(static_fraction=0.8, #
                                     desired_ffvector_len=1000,
                                     sigma=7.5,
                                     imshape=(414,414,25), #usualmente sería im.shape, 
                                     complex_output=True, # las imágenes de MRI son complejas
                                     key=key_B)

## Definir la función de pérdida 

In [ ]:
spclim = spacelim(X_train[:,1:,:]) # esto era para definir el dominio (en el kspace o en imagen, ya no me acuerdo)

short_spoke_loss = lambda params, train_X_sample, train_Y_sample: spoke_loss_fourierspace_phase(params, train_X_sample, train_Y_sample, spclim, 1., csmap, FFnet, 'ramp')
radon_fourier_loss_fourierspace_phase = jit(lambda params, train_X, train_Y: np.mean(vmap(short_spoke_loss, in_axes = (None, 0, 0))(params, train_X, train_Y)))
loss = lambda params, train_X, train_Y: radon_fourier_loss_fourierspace_phase(params, train_X, train_Y) 


## Entrenar la red 

In [ ]:
inner_layers = [512, 512, 512]

nIter = 1000 # 1k para probar, pero usualmente se necesita 10k, y sigue mejorando un poco hasta 20k
batch_size = 1 # lo maximo para una gpu de 10gb con esta configuracion de inner_layer y frac_static_mixed_params[1]
learning_rate = 1e-3 # TODO: ver cual usé 

params = FFnet.init_params(inner_layers, key = key_init)

optimizer = optax.adam(learning_rate)

results = simple_train(loss, *trainset, params,  optimizer, key_train, batch_size = batch_size, nIter = nIter)

In [ ]:
results.keys()

## Revisar resultados 

In [ ]:
plt.plot(results['iterations'], results['train_loss'])
plt.yscale('log')
plt.title('Evolución de la _loss_')

In [ ]:
frame = 8 

from jax.lax import map as laxmap 

def post_processing(im): # im.shape (px,py,nframes)
    is_inside = is_inside_of_radial_lim(FFnet._gridX, 1.)
    masks = (1 - hollow_mask) * is_inside
    return im * masks[:,:,None]
    

def vmap_image_pred(params):
    return vmap(FFnet.image_prediction_at_timeframe, in_axes=(None, 0), out_axes=-1)(params, FFnet._gridt) 

def laxmap_image_pred(params):
    reco = laxmap(lambda t: FFnet.image_prediction_at_timeframe(params, t), FFnet._gridt) 
    
    return np.moveaxis(reco, 0,-1)

from inrmri.utils import is_inside_of_radial_lim

# is_inside = is_inside_of_radial_lim()

fullreco = post_processing(laxmap_image_pred(results['last_param'])) # shape (414,414,25)

In [ ]:
fullreco.shape


In [ ]:
fig = plt.figure(figsize=(15,5))

plt.subplot(121)
# plt.imshow(np.abs(im[...,frame]), cmap='bone') # TODO 
plt.title(f"Reconstrucción completamente, \nmuestreada en el {frame}-ésimo frame")

plt.subplot(122)
plt.imshow(np.abs(fullreco[...,frame]), cmap='bone')
plt.title(f"Reconstrucción Neural Fields")

In [ ]:
plt.imshow(np.abs(fullreco)[:,fullreco.shape[1]//2,:].transpose(), cmap='bone')

## Calcular métricas 

## Métricas disponibles 

El diccionario `METRIC_FUNCTIONS` de `inrmri.metrics` tiene varias métricas implementadas. 
metrics_values 
- `'ssim_3D'`: a pesar del nombre, esta es la SSIM estándar, calculada sobre la imagen completa (como suelo trabajar con vídeos, mis imágenes son 3D y la llamada así para distinguirla de la siguiente)
- `'mean_ssim_2D'`: calcula la SSIM frame por frame y la promedia. Asume que las imagenes tienen shape `(px, py, total_frames)`.
- `'psnr'`
- `'ism_fsim'`, `'ism_issm'`, `'ism_psnr'`, `'ism_rmse'`: están directamente extraídas del paquete [https://pypi.org/project/image-similarity-measures/](https://pypi.org/project/image-similarity-measures/). No sé por qué `'ism_fsim'` no está funcionando con imágenes 2D.

## Preprocesamiento 

Suelo calcular las métricas en torno al corazón. La variable `PATIENTS_CROPS` de `inrmri.data_harvard` tiene calculadas las zonas adecuadas para varios pacientes. El preprocesamiento, que suele incluir corte, normalización y cálculo del valor absoluto, se realiza con `BeforeNormalizer` o `BeforeLinRegNormalizer`. El primero normaliza por la máxima intensidad del píxel en la zona cortada, el segundo escala la imagen por un factor adecuado de forma que maximize el PSNR, por lo que requiere la imagen de referencia.

In [ ]:
from inrmri.metrics import METRIC_FUNCTIONS 

print("The available metrics are: " + str(list(METRIC_FUNCTIONS.keys())))

from inrmri.image_processor import BeforeLinRegNormalizer, BeforeNormalizer 
from inrmri.data_harvard import PATIENTS_CROPS 

crop_ns = PATIENTS_CROPS[chosen_patient]
improc = BeforeLinRegNormalizer(im[...,frame], crop_ns)
# improc = BeforeNormalizer(crop_ns)

chosen_metrics = ['ssim_3D', 'psnr'] # no todas funcionan en 2D 

processed_reco = improc.process(fullreco)
processed_ref = improc.process(im)

metrics_values = {
    metric_name: METRIC_FUNCTIONS[metric_name](processed_ref, processed_reco) for metric_name in chosen_metrics
}

metrics_values 

In [ ]:

fig = plt.figure(figsize=(10,5))

plt.subplot(121)
plt.imshow(processed_ref, cmap='bone')
plt.title(f"Reconstrucción completamente, \nmuestreada en el {frame}-ésimo frame \n en la zona del corazón")

plt.subplot(122)
plt.imshow(processed_reco, cmap='bone')
metrics_str = " ".join([f"\n{metric_name} {metric_val:.2f}" for metric_name, metric_val in metrics_values.items()])
plt.title(f"Reconstrucción Neural Fields" + metrics_str)